In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy import stats
np.random.seed(123)  # for reproducibility

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import MaxPooling2D, Conv2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import keras.callbacks

import tensorflow as tf
# run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

/home/dp/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
model = load_model("checkpoints/weights.02-0.94.hdf5")

In [5]:
model.input_shape

(None, 256, 256, 3)

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 126, 126, 64)      18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 124, 124, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 64)        0         
__________

In [9]:
resol = 256
def preprocess(img):
    return cv2.bilateralFilter(img, 9, 75, 75)

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255,)
#                                  preprocessing_function=preprocess)
# test_generator = test_datagen.flo

In [11]:
test_generator = test_datagen.flow_from_directory("test_data/", target_size=(resol, resol),
                                                 batch_size=4, shuffle=False)

Found 2588 images belonging to 30 classes.


In [12]:
score = model.evaluate_generator(test_generator)

In [13]:
score

[0.9858297677379579, 0.9359093290023862]

In [14]:
test_generator.class_indices

{'antelope': 0,
 'bat': 1,
 'beaver': 2,
 'bobcat': 3,
 'buffalo': 4,
 'chihuahua': 5,
 'chimpanzee': 6,
 'collie': 7,
 'dalmatian': 8,
 'german+shepherd': 9,
 'grizzly+bear': 10,
 'hippopotamus': 11,
 'horse': 12,
 'killer+whale': 13,
 'mole': 14,
 'moose': 15,
 'mouse': 16,
 'otter': 17,
 'ox': 18,
 'persian+cat': 19,
 'raccoon': 20,
 'rat': 21,
 'rhinoceros': 22,
 'seal': 23,
 'siamese+cat': 24,
 'spider+monkey': 25,
 'squirrel': 26,
 'walrus': 27,
 'weasel': 28,
 'wolf': 29}

In [11]:
cname = {}  # mapping from index to class name
for cls, ind in test_generator.class_indices.items():
    cname[ind] = cls
cname

{0: 'antelope',
 1: 'bat',
 2: 'beaver',
 3: 'bobcat',
 4: 'buffalo',
 5: 'chihuahua',
 6: 'chimpanzee',
 7: 'collie',
 8: 'dalmatian',
 9: 'german+shepherd',
 10: 'grizzly+bear',
 11: 'hippopotamus',
 12: 'horse',
 13: 'killer+whale',
 14: 'mole',
 15: 'moose',
 16: 'mouse',
 17: 'otter',
 18: 'ox',
 19: 'persian+cat',
 20: 'raccoon',
 21: 'rat',
 22: 'rhinoceros',
 23: 'seal',
 24: 'siamese+cat',
 25: 'spider+monkey',
 26: 'squirrel',
 27: 'walrus',
 28: 'weasel',
 29: 'wolf'}

In [24]:
def get_class(impath):
    img = load_img(impath, target_size=(resol, resol))

    x = img_to_array(img)
    plt.imshow(np.asarray(img))
    x = x.reshape((1,) + x.shape) 
    prob = model.predict(x)
    classes = []
    for i, p in enumerate(prob[0]):
        if p > 0.5:
            classes.append(cname[i])
    
    return classes, prob